### Tercera etapa : Data preparation


Con los datos seleccionados del conjunto de datos, pasamos a generar nuevos datos adicionales que nos permitan entrenar el modelo con una cuantia de datos aun mayor dadas las clases seleccionadas, además de extraer nuevas características de nuestras imágemes para obtener mejores resultados con los modelos a usar. Es por ello que este apartado se divide en dos etapas:

1) Generación de imágenes derivadas de las actuales

2) Extracción de características de las imágenes

### Generación de nuevas imágenes

Para la generación de nuevas imágenes, se aplica la combinación de las siguientes técnicas:
- Rotación
- Escalado
- Traslación
- Volteo horizontal
- Cambios de brillo, contraste, saturación y gamma
- Adición de ruido gaussiano
- Aumentación Basada en Contexto Semántico

### Extracción de características

En cuanto a la extracción de nuevas características, estas se trataran de obtener haciendo uso de un modelo generativo, en este caso ____, de tal forma que nos permite obtener 

In [1]:
import cv2
import os
import numpy as np
import time
from tqdm import tqdm
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

ModuleNotFoundError: No module named 'segment_anything'